In [77]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

In [2]:
spark = SparkSession.builder.appName('Preprocessing').getOrCreate()

In [3]:
spark

In [114]:
data = spark.read.csv('test2.csv',header=True,inferSchema=True)

In [115]:
data.show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
|     null|  36|      null|  null|
+---------+----+----------+------+



In [116]:
data.select(['Name','age']).show()

+---------+----+
|     Name| age|
+---------+----+
|    Krish|  31|
|Sudhanshu|  30|
|    Sunny|  29|
|     Paul|  24|
|   Harsha|  21|
|  Shubham|  23|
|   Mahesh|null|
|     null|  34|
|     null|  36|
+---------+----+



In [117]:
data = data.withColumnRenamed('age','Age')

In [118]:
data.show()

+---------+----+----------+------+
|     Name| Age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
|     null|  36|      null|  null|
+---------+----+----------+------+



In [119]:
data.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [120]:
data.describe().show()

+-------+------+------------------+------------------+-----------------+
|summary|  Name|               Age|        Experience|           Salary|
+-------+------+------------------+------------------+-----------------+
|  count|     7|                 8|                 7|                8|
|   mean|  null|              28.5| 5.428571428571429|          25750.0|
| stddev|  null|5.3718844791323335|3.8234863173611093|9361.776388210581|
|    min|Harsha|                21|                 1|            15000|
|    max| Sunny|                36|                10|            40000|
+-------+------+------------------+------------------+-----------------+



In [121]:
data = data.na.drop(how='any',subset='Name')

In [122]:
data.show()

+---------+----+----------+------+
|     Name| Age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
+---------+----+----------+------+



In [123]:
data = data.na.fill(int(data.approxQuantile("Age", [0.95], 0.001)[0]),'Age')

In [124]:
from pyspark.ml.feature import Imputer

In [125]:
imputer = Imputer(strategy='median',inputCols=['Experience'],outputCols=[f'{c}_imputed' for c in ['Experience']])

In [137]:
imputer.fit(data).transform(data).drop('Experience').show()

+---------+---+------+------------------+
|     Name|Age|Salary|Experience_imputed|
+---------+---+------+------------------+
|    Krish| 31| 30000|                10|
|Sudhanshu| 30| 25000|                 8|
|    Sunny| 29| 20000|                 4|
|     Paul| 24| 20000|                 3|
|   Harsha| 21| 15000|                 1|
|  Shubham| 23| 18000|                 2|
|   Mahesh| 31| 40000|                14|
+---------+---+------+------------------+



In [127]:
data = data.na.fill(int(data.select(sum('Experience')).collect()[0][0]/2),'Experience')

In [128]:
data.describe().show()

+-------+------+-----------------+-----------------+-----------------+
|summary|  Name|              Age|       Experience|           Salary|
+-------+------+-----------------+-----------------+-----------------+
|  count|     7|                7|                7|                7|
|   mean|  null|             27.0|              6.0|          24000.0|
| stddev|  null|4.203173404306164|4.795831523312719|8582.928793055822|
|    min|Harsha|               21|                1|            15000|
|    max| Sunny|               31|               14|            40000|
+-------+------+-----------------+-----------------+-----------------+



In [129]:
data.filter(data['Name'] == 'Krish').show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Krish| 31|        10| 30000|
+-----+---+----------+------+



In [130]:
data.filter(~(data['Name'] == 'Krish')).show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
|   Mahesh| 31|        14| 40000|
+---------+---+----------+------+



In [136]:
data.filter((data['Salary'] > 18000) & (data['Salary'] < 30000)).select('Name','Salary').show()

+---------+------+
|     Name|Salary|
+---------+------+
|Sudhanshu| 25000|
|    Sunny| 20000|
|     Paul| 20000|
+---------+------+

